In [6]:
import cv2
import os
import numpy as np
import albumentations as A
from tqdm import tqdm
import multiprocessing
from functools import partial

class DatasetAugmentor:
    def __init__(self, source_dir='large_dataset', target_dir='augmented_dataset'):
        self.source_dir = source_dir
        self.target_dir = target_dir
        self.augmentations = self.get_augmentation_pipeline()
        self.augmentation_factor = 10  # Create 10 augmented versions of each image
        
    def get_augmentation_pipeline(self):
        """Create comprehensive augmentation pipeline"""
        return A.Compose([
            # Geometric transformations
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=15, p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.5),
            
            # Brightness/contrast variations
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.3),
            
            # Blur and noise
            A.GaussianBlur(blur_limit=3, p=0.3),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            
            # Perspective and distortion
            A.Perspective(scale=(0.05, 0.1), p=0.3),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.3),
            
            # Weather effects (simulated)
            A.RandomGamma(gamma_limit=(80, 120), p=0.2),
            A.ChannelShuffle(p=0.1),
            
            # Crop and scale
            A.RandomCrop(height=44, width=44, p=0.3),
            A.Resize(height=48, width=48)
        ])
    
    def augment_single_image(self, image_path, output_dir, expression, aug_index):
        """Augment a single image"""
        try:
            # Read image
            image = cv2.imread(image_path)
            if image is None:
                return
            
            # Convert BGR to RGB for albumentations
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Apply augmentations
            augmented = self.augmentations(image=image)
            augmented_image = augmented['image']
            
            # Convert back to BGR for OpenCV
            augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
            
            # Save augmented image
            filename = os.path.basename(image_path)
            name, ext = os.path.splitext(filename)
            output_path = os.path.join(output_dir, f"{name}_aug{aug_index:02d}{ext}")
            
            cv2.imwrite(output_path, augmented_image)
            
        except Exception as e:
            print(f"❌ Error augmenting {image_path}: {e}")
    
    def augment_expression(self, expression):
        """Augment all images for a specific expression"""
        source_expr_dir = os.path.join(self.source_dir, expression)
        target_expr_dir = os.path.join(self.target_dir, expression)
        
        if not os.path.exists(source_expr_dir):
            print(f"❌ Source directory not found: {source_expr_dir}")
            return 0
        
        # Create target directory
        os.makedirs(target_expr_dir, exist_ok=True)
        
        # Get all image files
        image_files = []
        for ext in ['*.jpg', '*.jpeg', '*.png']:
            image_files.extend([f for f in os.listdir(source_expr_dir) if f.lower().endswith(ext[1:])])
        
        if not image_files:
            print(f"❌ No images found for {expression}")
            return 0
        
        print(f"🔄 Augmenting {expression}: {len(image_files)} images")
        
        # Copy original images first
        for img_file in image_files:
            src_path = os.path.join(source_expr_dir, img_file)
            dst_path = os.path.join(target_expr_dir, img_file)
            if not os.path.exists(dst_path):
                import shutil
                shutil.copy2(src_path, dst_path)
        
        # Augment images
        total_augmented = 0
        for aug_index in range(self.augmentation_factor):
            print(f"   Augmentation round {aug_index + 1}/{self.augmentation_factor}")
            
            for img_file in tqdm(image_files, desc=f"  {expression}"):
                img_path = os.path.join(source_expr_dir, img_file)
                self.augment_single_image(img_path, target_expr_dir, expression, aug_index)
                total_augmented += 1
        
        original_count = len(image_files)
        augmented_count = original_count * (self.augmentation_factor + 1)  # +1 for originals
        
        print(f"✅ {expression}: {original_count} → {augmented_count} images")
        return augmented_count
    
    def augment_all(self):
        """Augment all expressions"""
        print("🚀 STARTING DATASET AUGMENTATION")
        print("=" * 50)
        
        expressions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
        
        total_original = 0
        total_augmented = 0
        
        for expression in expressions:
            count = self.augment_expression(expression)
            total_augmented += count
            
            # Count originals
            source_expr_dir = os.path.join(self.source_dir, expression)
            if os.path.exists(source_expr_dir):
                original_files = []
                for ext in ['*.jpg', '*.jpeg', '*.png']:
                    original_files.extend([f for f in os.listdir(source_expr_dir) if f.lower().endswith(ext[1:])])
                total_original += len(original_files)
        
        print("\n" + "=" * 50)
        print("📊 AUGMENTATION COMPLETE")
        print(f"📈 Original images: {total_original:,}")
        print(f"🎯 Augmented images: {total_augmented:,}")
        print(f"📈 Total multiplier: {total_augmented/total_original:.1f}x")
        print(f"💾 Augmented dataset saved to: {self.target_dir}")

def main():
    print("🎭 DATASET AUGMENTATION TOOL")
    print("This will create augmented versions of your dataset for better training")
    
    # Install albumentations if not available
    try:
        import albumentations
    except ImportError:
        print("📦 Installing albumentations...")
        import subprocess
        subprocess.check_call(['pip', 'install', 'albumentations'])
        import albumentations
    
    augmentor = DatasetAugmentor()
    augmentor.augment_all()

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'albumentations'

In [2]:
!pip install albumentations

   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.9 MB ? eta -:--:--
    --------------------------------------- 0.5/38.9 MB 578.7 kB/s eta 0:01:07
    --------------------------------------- 0.5/38.9 MB 578.7 kB/s eta 0:01:07
    --------------------------------------- 0.8/38.9 MB 699.0 kB/s eta 0:00:55
   - -------------------------------------- 1.3/38.9 MB 932.1 kB/s eta 0:00:41
   - -------------------------------------- 1.3/38.9 MB 932.1 kB/s eta 0:00:41
   - -------------------------------------- 1.6/38.9 MB 953.2 kB/s eta 0:00:40
   - -------------------------------------- 1.8/38.9 MB 958.5 kB/s eta 0:00:39
   -- ------------------------------------- 2.1/38.9 MB 987.1 kB/s eta 0:00:38
   -- ------------------------------------- 2.4/38.9 MB 1.0 MB/s eta 0:00:36
   -- -----

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\vansh\\anaconda3\\envs\\env\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [5]:
!pip install albumentations==1.3.1

  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl (38.9 MB)
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
    --------------------------------------- 0.3/12.8 MB ? eta -:--:--
    --------------------------------------- 0.3/12.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.8 MB 699.0 kB/s eta 0:00:18
   - -------------------------------------- 0.5/12.8 MB 699.0 kB/s eta 0:00:18
   -- ------------------------------------- 0.8/12.8 MB 713.3 kB/s eta 0:00:17
   --- ------------------------------------ 1.0/12.8 MB 708.5 kB/s eta 0:00:17
   --- --------------------------------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\vansh\\anaconda3\\envs\\env\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.

